<a href="https://colab.research.google.com/github/Invalid07/kaggle_bag_comp/blob/main/bag_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
!smi-nvidia


/bin/bash: line 1: smi-nvidia: command not found


In [3]:
# kaggle
! mkdir -p /root/.kaggle
! mv kaggle.json /root/.kaggle
import kaggle

In [4]:
!kaggle competitions download -c playground-series-s5e2

In [5]:
! unzip playground-series-s5e2.zip

Archive:  playground-series-s5e2.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
  inflating: training_extra.csv      


In [6]:
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")
extra_train_df = pd.read_csv("/content/training_extra.csv")
sample_submission = pd.read_csv("/content/sample_submission.csv")





In [7]:
test_df.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,300000,Puma,Leather,Small,2.0,No,No,Tote,Green,20.671147
1,300001,Nike,Canvas,Medium,7.0,No,Yes,Backpack,Green,13.564105
2,300002,Adidas,Canvas,Large,9.0,No,Yes,Messenger,Blue,11.809799
3,300003,Adidas,Nylon,Large,1.0,Yes,No,Messenger,Green,18.477036
4,300004,NaN,Nylon,Large,2.0,Yes,Yes,Tote,Black,9.907953


In [8]:
# Display first few rows
train_df.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    300000 non-null  int64  
 1   Brand                 290295 non-null  object 
 2   Material              291653 non-null  object 
 3   Size                  293405 non-null  object 
 4   Compartments          300000 non-null  float64
 5   Laptop Compartment    292556 non-null  object 
 6   Waterproof            292950 non-null  object 
 7   Style                 292030 non-null  object 
 8   Color                 290050 non-null  object 
 9   Weight Capacity (kg)  299862 non-null  float64
 10  Price                 300000 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 25.2+ MB


In [10]:
train_df.isnull().sum()

,0
id,0
Brand,9705
Material,8347
Size,6595
Compartments,0
Laptop Compartment,7444
Waterproof,7050
Style,7970
Color,9950
Weight Capacity (kg),138


In [11]:
len(train_df)

300000

In [12]:
train_df.duplicated().sum()


np.int64(0)

In [13]:
brand=train_df["Brand"].unique()
brand

array(['Jansport', 'Under Armour', 'Nike', 'Adidas', 'Puma', nan],
      dtype=object)

In [14]:
l=train_df.columns

In [15]:
for i in train_df.columns:
    print("-->", train_df[i].unique())

--> [     0      1      2 ... 299997 299998 299999]
--> ['Jansport' 'Under Armour' 'Nike' 'Adidas' 'Puma' nan]
--> ['Leather' 'Canvas' 'Nylon' nan 'Polyester']
--> ['Medium' 'Small' 'Large' nan]
--> [ 7. 10.  2.  8.  1.  3.  5.  9.  6.  4.]
--> ['Yes' 'No' nan]
--> ['No' 'Yes' nan]
--> ['Tote' 'Messenger' nan 'Backpack']
--> ['Black' 'Green' 'Red' 'Blue' 'Gray' 'Pink' nan]
--> [11.61172281 27.07853658 16.64375995 ...  9.55993494 26.63318223
  6.1757379 ]
--> [112.15875  68.88056  39.1732  ...  78.7574  131.37288  41.96325]


In [16]:
null_rows = train_df[train_df['Brand'].isnull()]
null_rows

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
30,30,NaN,Leather,Large,3.0,No,Yes,Tote,Blue,21.304434,138.32699
48,48,NaN,Canvas,Medium,2.0,No,No,Tote,NaN,5.000000,42.28337
90,90,NaN,Leather,Medium,8.0,Yes,No,Tote,Blue,16.594824,142.11363
125,125,NaN,Polyester,Small,8.0,No,No,Backpack,Red,7.922756,64.32242
137,137,NaN,Nylon,Medium,3.0,Yes,No,Backpack,Green,13.051134,34.86676
...,...,...,...,...,...,...,...,...,...,...,...
299921,299921,NaN,NaN,Small,7.0,No,Yes,Tote,NaN,20.419934,27.58667
299937,299937,NaN,Polyester,Large,8.0,No,Yes,Messenger,Gray,19.272669,74.70298
299957,299957,NaN,Polyester,Large,9.0,No,Yes,Tote,Black,13.619044,19.25284
299978,299978,NaN,Canvas,Large,6.0,Yes,No,Tote,Gray,5.000000,50.78781


In [17]:
pd.set_option('display.max_rows', None)

In [18]:
train_df.columns

Index(['id', 'Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price'],
      dtype='object')

In [19]:
import pandas as pd

def drop_null_brand_material(df, inplace=False):
    # Drop rows where both 'Brand' and 'Material' are null
    if inplace:
        df.dropna(subset=['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price'], how='all', inplace=True)
    else:
        df = df.dropna(subset=['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price'], how='all')
    return df


In [20]:
train_df = drop_null_brand_material(train_df, inplace=True)  # This will modify the original DataFrame in place


In [21]:
train_df.isnull().sum()


,0
id,0
Brand,9705
Material,8347
Size,6595
Compartments,0
Laptop Compartment,7444
Waterproof,7050
Style,7970
Color,9950
Weight Capacity (kg),138


In [22]:
len(train_df)

300000

In [23]:
train_df.head(10)

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312
5,5,Nike,Canvas,Medium,10.0,No,Yes,NaN,Black,7.241812,20.01553
6,6,Nike,NaN,Large,3.0,No,No,Backpack,Green,6.828123,84.80500
7,7,Puma,Canvas,Small,1.0,Yes,Yes,Backpack,Blue,21.488864,27.15815
8,8,Under Armour,Polyester,Medium,8.0,Yes,No,Tote,Gray,10.207780,25.98652
9,9,Under Armour,Nylon,Medium,2.0,Yes,Yes,Messenger,Pink,15.895100,38.48741


In [24]:
# brand

In [25]:
train_df[train_df['Brand']=='Material']

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price


In [26]:


brand_style_waterproof = train_df.groupby(['Brand', 'Style', 'Waterproof'])['Brand'].count().reset_index(name='Count')


In [27]:
brand_style_waterproof

,Brand,Style,Waterproof,Count
0,Adidas,Backpack,No,9064
1,Adidas,Backpack,Yes,9340
2,Adidas,Messenger,No,9874
3,Adidas,Messenger,Yes,9932
4,Adidas,Tote,No,9426
5,Adidas,Tote,Yes,9530
6,Jansport,Backpack,No,8729
7,Jansport,Backpack,Yes,8569
8,Jansport,Messenger,No,8876
9,Jansport,Messenger,Yes,9206


In [28]:
# replace all nan

In [29]:
for i in train_df.columns:
    if train_df[i].dtype == 'object':  # Check data type
        train_df[i] = train_df[i].fillna(train_df[i].mode()[0])  # Fill with mode for categorical columns
    else:
        train_df[i] = train_df[i].fillna(train_df[i].mean())  # Fill with mean for numerical columns

In [30]:
train_df.isnull().sum()

,0
id,0
Brand,0
Material,0
Size,0
Compartments,0
Laptop Compartment,0
Waterproof,0
Style,0
Color,0
Weight Capacity (kg),0


In [31]:
from sklearn.preprocessing import OneHotEncoder

categorical_features = ['Brand', 'Style', 'Waterproof', 'Material', 'Size', 'Compartments', 'Laptop Compartment', 'Color']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse=False for dense output
encoded_data = encoder.fit_transform(train_df[categorical_features])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_features))

train_df = train_df.drop(categorical_features, axis=1) # Drop original categorical columns
train_df = pd.concat([train_df, encoded_df], axis=1) # Concatenate encoded features

In [32]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_df['Weight Capacity (kg)'] = scaler.fit_transform(train_df[['Weight Capacity (kg)']])
train_df['Weight Capacity (kg)'] = scaler.fit_transform(train_df['Weight Capacity (kg)'].values.reshape(-1, 1))


In [33]:
from sklearn.model_selection import train_test_split

X = train_df.drop('Price', axis=1)  # Features
y = train_df['Price']  # Target variable

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
import tensorflow as tf

from tensorflow.keras.layers import Dropout

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    # Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    # Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    # Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])




from tensorflow.keras.optimizers import Adam
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae','accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
# del model

In [37]:
history = model.fit(X_train, y_train, epochs=5, batch_size=10, validation_data=(X_val, y_val))

Epoch 1/5
24000/24000 ━━━━━━━━━━━━━━━━━━━━ 75s 3ms/step - accuracy: 0.0000e+00 - loss: 8013.3799 - mae: 80.5429 - val_accuracy: 0.0000e+00 - val_loss: 7958.5298 - val_mae: 80.2616
Epoch 2/5
24000/24000 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.0000e+00 - loss: 7994.4053 - mae: 80.4255 - val_accuracy: 0.0000e+00 - val_loss: 7958.5298 - val_mae: 80.2616
Epoch 3/5
24000/24000 ━━━━━━━━━━━━━━━━━━━━ 73s 3ms/step - accuracy: 0.0000e+00 - loss: 8010.0308 - mae: 80.5349 - val_accuracy: 0.0000e+00 - val_loss: 7958.5298 - val_mae: 80.2616
Epoch 4/5
24000/24000 ━━━━━━━━━━━━━━━━━━━━ 81s 3ms/step - accuracy: 0.0000e+00 - loss: 7995.3477 - mae: 80.4276 - val_accuracy: 0.0000e+00 - val_loss: 7958.5298 - val_mae: 80.2616
Epoch 5/5
24000/24000 ━━━━━━━━━━━━━━━━━━━━ 74s 3ms/step - accuracy: 0.0000e+00 - loss: 7988.5088 - mae: 80.4044 - val_accuracy: 0.0000e+00 - val_loss: 7958.5298 - val_mae: 80.2616


In [39]:
# Evaluate on validation set
mae = model.evaluate(X_val, y_val, verbose=0)
print("Validation MAE:", mae)

# Make predictions on the test set (after preprocessing it similarly)
predictions = model.predict(X_val)
predictions

Validation MAE: [7958.55029296875, 80.26150512695312, 0.0]
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [ ]:
# model.save(filepath='C:\\Users\\Vikas\\jupyter book\\.ipynb_checkpoints\\my_model.keras')

In [49]:
# hyper tunning
!pip install scikeras
!pip install --upgrade scikeras

In [50]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

def create_model(learning_rate=0.01, neurons=64):

  model = tf.keras.models.Sequential([
           tf.keras.layers.Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
           tf.keras.layers.Dense(neurons // 2, activation='relu'),
           tf.keras.layers.Dense(1, activation='sigmoid')
       ])

  optimizer = Adam(learning_rate=learning_rate)
  model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae','accuracy'])
  return model

model = KerasRegressor(build_fn=create_model, verbose=0)

param_grid = {
       'learning_rate': [0.01, 0.001, 0.0001],
       'neurons': [64, 128, 256],
       'batch_size': [32, 64],
       'epochs': [7, 10]
   }
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [53]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# ... (Your create_model function remains the same) ...

# Wrap the model
model = KerasRegressor(model=create_model, verbose=0)

# Define a smaller parameter grid initially
param_grid = {
    'model__learning_rate': [0.01, 0.001],
    'model__neurons': [64, 128],
    'batch_size': [32],
    'epochs': [7]
}

# Use GridSearchCV with reduced resource demand
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=2, cv=2)  # n_jobs=1
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

AttributeError: 'super' object has no attribute '__sklearn_tags__'